<a href="https://colab.research.google.com/github/paradisepilot/study/blob/master/earth-observation/earth-engine/e2e-gee/e2e_gee_06_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=EW4IttZ-1Y6me6ozV90q4Euy2ottSPHqOCtx_T6aNEA&tc=tcrCuJca73OGZK4tzG-QOSlcM7sH1XLfP5nezrX0p_s&cc=TldI33dy1jdhdnzQSjrrupm41MCTcRKPz5kkyK6QWJ8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjcox_OdQLIfwDwbDiN7ds4RsbhacFMkWzCUUIEWVj20t9mp_A6IGM

Successfully saved authorization token.


In [3]:
ee.Initialize()


In [4]:
city = 'San Fransico'
state = 'California'
print(city, state)

population = 881549
print(population)

San Fransico California
881549


In [5]:
lon_centre = -75.695000
lat_centre =  45.424722 

lon_width  = 0.50
lat_width  = 0.30

ottawaGeometry = ee.Geometry.Polygon([[
  [lon_centre - lon_width/2, lat_centre - lat_width/2],
  [lon_centre - lon_width/2, lat_centre + lat_width/2],
  [lon_centre + lon_width/2, lat_centre + lat_width/2],
  [lon_centre + lon_width/2, lat_centre - lat_width/2],
  [lon_centre - lon_width/2, lat_centre - lat_width/2]
]])

s2 = ee.ImageCollection('COPERNICUS/S2')

In [6]:
filtered = s2 \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .filter(ee.Filter.date('2022-02-01','2022-03-01')) \
    .filter(ee.Filter.bounds(ottawaGeometry))

In [7]:
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image,["system:time_start"])

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered \
    .map(maskS2clouds) \
    .map(addNDVI)


In [8]:
composite = withNdvi.median()
ndvi = composite.select('ndvi')

stats = ndvi.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': ottawaGeometry,
  'scale': 10,
  'maxPixels': 1e10
  })

print(stats.get('ndvi').getInfo())


0.02846762328458271


In [9]:
# javascript:
# var myList = ee.List.sequence(1, 10);
# var newList = myList.map(function(number) {
#     return ee.Number(number).pow(2);
#     });
# print(newList);

myList = ee.List.sequence(1, 10)
newList = myList.map(lambda number: ee.Number(number).pow(2))
print(newList.getInfo())


[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [10]:
# Exercise
geometry = ee.Geometry.Point([77.60412933051538, 12.952912912328241]);

s2 = ee.ImageCollection('COPERNICUS/S2');

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2019-01-01','2020-01-01')) \
  .filter(ee.Filter.bounds(geometry));
  
print(filtered.size().getInfo());

30
